In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, SimpleRNN, GRU
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [19]:
#import wandb
##run = wandb.init()
##config = run.config
config_hidden_nodes = 128
config_batch_size = 256
config_file = 'male.txt'
config_maxlen = 200
config_step = 3

In [62]:
text = io.open(config_file, encoding='utf-8').read()
##text = text.replace('\n', ' ')
chars = sorted(list(set(text)))

In [63]:
len(chars)

54

In [64]:
len(text)/3

6764.333333333333

In [65]:
chars[0:10]

['\n', '-', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [66]:
text[0:10]

'Aamir\nAaro'

In [67]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [68]:
len(text)

20293

In [69]:
sentences = []
next_chars = []
for i in range(0, len(text) - config_maxlen, config_step):
    sentences.append(text[i: i + config_maxlen])
    next_chars.append(text[i + config_maxlen])

In [70]:
sentences[0:3]

['Aamir\nAaron\nAbbey\nAbbie\nAbbot\nAbbott\nAbby\nAbdel\nAbdul\nAbdulkarim\nAbdullah\nAbe\nAbel\nAbelard\nAbner\nAbraham\nAbram\nAce\nAdair\nAdam\nAdams\nAddie\nAdger\nAditya\nAdlai\nAdnan\nAdolf\nAdolfo\nAdolph\nAdolphe\nAdolpho\nA',
 'ir\nAaron\nAbbey\nAbbie\nAbbot\nAbbott\nAbby\nAbdel\nAbdul\nAbdulkarim\nAbdullah\nAbe\nAbel\nAbelard\nAbner\nAbraham\nAbram\nAce\nAdair\nAdam\nAdams\nAddie\nAdger\nAditya\nAdlai\nAdnan\nAdolf\nAdolfo\nAdolph\nAdolphe\nAdolpho\nAdol',
 'Aaron\nAbbey\nAbbie\nAbbot\nAbbott\nAbby\nAbdel\nAbdul\nAbdulkarim\nAbdullah\nAbe\nAbel\nAbelard\nAbner\nAbraham\nAbram\nAce\nAdair\nAdam\nAdams\nAddie\nAdger\nAditya\nAdlai\nAdnan\nAdolf\nAdolfo\nAdolph\nAdolphe\nAdolpho\nAdolphu']

In [71]:
next_chars[0:3]

['d', 'p', 's']

In [72]:
x = np.zeros((len(sentences), config_maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [73]:
x.shape

(6698, 200, 54)

In [74]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



In [75]:
x[1]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [76]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [77]:
model = Sequential()
model.add(SimpleRNN(128, input_shape=(config_maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="rmsprop")

In [78]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [79]:
class SampleText(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        start_index = random.randint(0, len(text) - config_maxlen - 1)

        for diversity in [0.5, 1.2]:
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + config_maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(200):
                x_pred = np.zeros((1, config_maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [80]:
model.fit(x, y, batch_size=config_batch_size,
              epochs=100, callbacks=[SampleText()])

Epoch 1/100
6698/6698 [==============================] - 6s 888us/step - loss: 3.3747

----- diversity: 0.5
----- Generating with seed: "ton
Remus
Renado
Renaldo
Renard
Renato
Renaud
Renault
Rene
Reube
Reuben
Reuven
Rex
Rey
Reynard
Reynold
Reynolds
Reza
Rhett
Ric
Ricard
Ricardo
Riccardo
Rice
Rich
Richard
Richardo
Richie
Richmond
Richy
"
ton
Remus
Renado
Renaldo
Renard
Renato
Renaud
Renault
Rene
Reube
Reuben
Reuven
Rex
Rey
Reynard
Reynold
Reynolds
Reza
Rhett
Ric
Ricard
Ricardo
Riccardo
Rice
Rich
Richard
Richardo
Richie
Richmond
Richy



e
ll


i




a

rt


m






nu

Ry

na

M

l


e



S


M






rir


oJn



o





n



rl










a








e




l
e











s








ue



















ta


irl



l

l

ss



r
sa




----- diversity: 1.2
----- Generating with seed: "ton
Remus
Renado
Renaldo
Renard
Renato
Renaud
Renault
Rene
Reube
Reuben
Reuven
Rex
Rey
Reynard
Reynold
Reynolds
Reza
Rhett
Ric
Ricard
Ricardo
Riccardo
Rice
Rich
Richard
Richardo
Richie
Richmond
Richy
"
ton
Rem

Pev
Dencou
ReVLnor
Rill-rqgtn
SrertsOSsuM
Viel
dtlhgeswYEYgi
Tedge
RocbaeRedRF
Reyur
CdoTeuJd
SAmmQyusom
BeenyBScsbnPqiesPaafTIDvilk
Qutyy
Khanmtrt
AeoNlzp
BMytgKan
AhpnsnioV
CFivy
Miase
DreneJ
FdegtVnf
Epoch 13/100
6698/6698 [==============================] - 6s 856us/step - loss: 2.1550

----- diversity: 0.5
----- Generating with seed: "re
Waring
Warner
Warren
Wash
Washington
Wat
Waverley
Waverly
Way
Waylan
Wayland
Waylen
Waylin
Waylon
Wayne
Web
Webb
Weber
Webster
Weidar
Weider
Welbie
Welby
Welch
Wells
Welsh
Wendall
Wendel
Wendell
We"
re
Waring
Warner
Warren
Wash
Washington
Wat
Waverley
Waverly
Way
Waylan
Wayland
Waylen
Waylin
Waylon
Wayne
Web
Webb
Weber
Webster
Weidar
Weider
Welbie
Welby
Welch
Wells
Welsh
Wendall
Wendel
Wendell
Wellen
Dullen
Clan
Roat
Donrro
Aoery
carre
Gorder
Gurtere
Lerdee
Haner
Laun
Aneee
Jen
Jeon
HesGord
Werwor
Eedere
Tee
Jerse
Jhere
Marfe
Ponce
Benfarer
Aaderril
Jerric
Jerrine
Perrr
Paafar
Britare
Raale
J

----- diversity: 1.2
----- Generating with seed: "re
Wa

ClanctrslastiscElminHylvaheZ
SCalhsay
WakeVistgl
nhjrshierOnoCnHajrindoH
-loodlie
FozThertaC
HavtsWllfy
Filmy
AmmledpeuZelQNmKpxTVNldrrtol
FCrtoss
ApyhuM
MiibkMohtmEMkhmQrkMfy
Ghdkesrr
Kston
Sesces
AtaOw
Epoch 19/100
6698/6698 [==============================] - 6s 843us/step - loss: 1.9013

----- diversity: 0.5
----- Generating with seed: "l
Mitchell
Moe
Mohamad
Mohamed
Mohammad
Mohammed
Mohan
Moise
Moises
Moishe
Monroe
Montague
Monte
Montgomery
Monty
Moore
Mordecai
Morgan
Morlee
Morley
Morly
Morrie
Morris
Morry
Morse
Mort
Morten
Mortie"
l
Mitchell
Moe
Mohamad
Mohamed
Mohammad
Mohammed
Mohan
Moise
Moises
Moishe
Monroe
Montague
Monte
Montgomery
Monty
Moore
Mordecai
Morgan
Morlee
Morley
Morly
Morrie
Morris
Morry
Morse
Mort
Morten
Mortie
Marnil
Marcil
Martil
Martil
Mirtol
Mertil
Mertil
Mertil
Maris
Martil
Martil
Martil
Barbil
Bauril
Paruil
Parri
Mareri
Marey
Marte
Marted
Martie
Martis
Martin
Marnil
Marril
Marril
Mardil
Mardil
Martil


----- diversity: 1.2
----- Generating with seed: "l
Mi

Allritu
Sporie
AAlcher
Chuch
PaefhaPzlkM
xadk
NhgougZ
VoWeln
XvineerDsRnshebthrim
Rucl
Nunninhel
TbwpsdbuiSk
Fei
SehwdytoU
Saldorac
OoreaDKheylle
PiSloe
Sixle
Fayly
Hangm
Laenderk
FnnTmwnrnHs
phebUnwBTd
Epoch 25/100
6698/6698 [==============================] - 6s 938us/step - loss: 1.6914

----- diversity: 0.5
----- Generating with seed: "il
Philbert
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter
Pietro
Piggy
Pincas
Pinchas
Pincus
Piotr
Pip
Plato
Pooh
Porter
Poul
Powell
Praneetf
Prasad
Prasun
Prent
Pren"
il
Philbert
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter
Pietro
Piggy
Pincas
Pinchas
Pincus
Piotr
Pip
Plato
Pooh
Porter
Poul
Powell
Praneetf
Prasad
Prasun
Prent
Prened
Moree
Morhed
Gordie
Gordon
Roddon
Rmdir
Rodd
Roddoe
Roddo
Roddory
Rodron
Redrrin
Georin
Rever
Reke
Bertia
Peres
Pherted
Peeree
Pane
Parle
Perrce
Perrce
Gedrord
Goore
Gerre
Gord
Gerdie
Gerdon
Geod
G

----- diversity: 1.2
----- Generating with seed: "il
Ph

Gregorian
MRudarto
Gouria
AuGovr
RuddonggTRmonmmu
Rodne
Rgor
todoeo
Eario
Todd
Rlmogd
Rodd
Rood
HyroRd
IruoeaToo
Dorio
OonaurU
Ranhigar
RonosoJunusel
Jegus
Aus
Bnuho
Bend
Reando
Docel
AuimsmGz
RmiuYon
Rimie

Epoch 31/100
6698/6698 [==============================] - 8s 1ms/step - loss: 1.5100

----- diversity: 0.5
----- Generating with seed: "h
Enrico
Enrique
Ephraim
Ephram
Ephrayim
Ephrem
Er
Erasmus
Erastus
Erek
Erhard
Erhart
Eric
Erich
Erick
Erik
Erin
Erl
Ernest
Ernesto
Ernie
Ernst
Erny
Errol
Ervin
Erwin
Esau
Esme
Esteban
Ethan
Ethelbert"
h
Enrico
Enrique
Ephraim
Ephram
Ephrayim
Ephrem
Er
Erasmus
Erastus
Erek
Erhard
Erhart
Eric
Erich
Erick
Erik
Erin
Erl
Ernest
Ernesto
Ernie
Ernst
Erny
Errol
Ervin
Erwin
Esau
Esme
Esteban
Ethan
Ethelbert
Hamber
Waldor
Wildor
Wildoris
Tilo
Wallie
Tanbare
Frercer
Gerrere
Grrreer
Grreed
Grreerer
Gerrerker
Gerrie
Gerrie
Gerrie
Gerrie
GerrieGerriG
Grrrer
Grrtere
Grrree
Grree
Gerre
Gerron
Gerry
Gerrce
FGrr

----- diversity: 1.2
----- Generating with seed: "h


...

TertynondUe

Jamxwoa
Jo
Qainh
Bainh
Dsiat
Muskef
MuNfg
PMike
Maddy
MeysoNh
Myerla
Maubain
cay
Shuwie
ACwhmd
SphAhbiht
ApstEn
Anguen
AryE
Aiure
EoAffHod
AhffeAddEtAne
Addy
Eudaod
Elparaue
ACorten
Arrse

Epoch 73/100
6698/6698 [==============================] - 6s 838us/step - loss: 0.7060

----- diversity: 0.5
----- Generating with seed: "briel
Gabriele
Gabriell
Gabriello
Gail
Gale
Galen
Gallagher
Gamaliel
Garcia
Garcon
Gardener
Gardiner
Gardner
Garey
Garfield
Garfinkel
Garold
Garp
Garret
Garrett
Garrot
Garrott
Garry
Garth
Garv
Garvey
"
briel
Gabriele
Gabriell
Gabriello
Gail
Gale
Galen
Gallagher
Gamaliel
Garcia
Garcon
Gardener
Gardiner
Gardner
Garey
Garfield
Garfinkel
Garold
Garp
Garret
Garrett
Garrot
Garrott
Garry
Garth
Garv
Garvey
Pest
PeetPet
PePereeP
Peffare
Le
Paire
Plaste
Pevas
Pete
Peele
Peteree

Perey
Le
Parle
Pearle
Pearre
He
Brie
Herrie
Herrin
HerFren
Herrin
Hesarres
Herres
Herrer
Herret
Herry
Hessere
Heeral
Hesalle
Hal

----- diversity: 1.2
----- Generating with seed: "briel


Bennett
Bincorve
Borroces
Pudpled
Jeyn
Mofff
Jefff
Jeto
Jopoueo
Aubeef
Ahavin
Masuitho
Phlisas
Mathas
Marfis
aarloy
Cay
Toyazeu
Nace
Boldias
Tradmrt
Tamode
WalietW
Bannoras
Oase
Te
Handdlo
Paby
Taie
Tos
Teos
Epoch 79/100
6698/6698 [==============================] - 6s 840us/step - loss: 0.6412

----- diversity: 0.5
----- Generating with seed: "mer
Wilson
Wilt
Wilton
Win
Windham
Winfield
Winford
Winfred
Winifield
Winn
Winnie
Winny
Winslow
Winston
Winthrop
Winton
Wit
Witold
Wittie
Witty
Wojciech
Wolf
Wolfgang
Wolfie
Wolfram
Wolfy
Woochang
Woo"
mer
Wilson
Wilt
Wilton
Win
Windham
Winfield
Winford
Winfred
Winifield
Winn
Winnie
Winny
Winslow
Winston
Winthrop
Winton
Wit
Witold
Wittie
Witty
Wojciech
Wolf
Wolfgang
Wolfie
Wolfram
Wolfy
Woochang
Woon
Wionerlon
Reorrin
Gorry
Gooren
Gobrin
Gefrundees
Rebeeth

Weldy
Dodo
Ranl
Mon
Wallor
WilloncWilk
Frickil
Wiley
Wilo
Wislo
Wilmon
Wickie
Wicki
Wildyi
Wildii
Wildin
Wilo
Rillis
Rorniel
Runnel
Runein
R

----- diversity: 1.2
----- Generating with seed: "

Wood
negles
Tereo
Le
Perrane
Mrrcer
Grerse
Grres
Gerree
Gerrgie
Logrinoe
Quiner
Aonper
Arnuse
Autut
Autustan
Aus
Anvew
My
Jespl
Jeds
Jofnweo
Brchore
Burthernghre
Brron
Garr
Gemvicung
GGvffeusy
Jush
Jdyai

Epoch 85/100
6698/6698 [==============================] - 6s 958us/step - loss: 0.5861

----- diversity: 0.5
----- Generating with seed: "
Kelsey
Kelvin
Kelwin
Ken
Kendal
Kendall
Kendrick
Kenn
Kennedy
Kenneth
Kenny
Kent
Kenton
Kenyon
Kermie
Kermit
Kerry
Kevan
Kevin
Kim
Kimball
Kimmo
Kin
Kincaid
King
Kingsley
Kingsly
Kingston
Kip
Kirby
K"

Kelsey
Kelvin
Kelwin
Ken
Kendal
Kendall
Kendrick
Kenn
Kennedy
Kenneth
Kenny
Kent
Kenton
Kenyon
Kermie
Kermit
Kerry
Kevan
Kevin
Kim
Kimball
Kimmo
Kin
Kincaid
King
Kingsley
Kingsly
Kingston
Kip
Kirby
Kilfy
Iell
Cyin
Clitin
Silmyig
Simdyiin
Sied
Cytyn
erty
Denerr
Dan
urice
Durgen
Gaiserer
Genrare
Granre
Gerrcem
Gerrend
Gerrad
Gerace
Herrame
Hurres
Harse
Harast
Haray
Hacaire
Habres
Harsy
Haraet
Harne

----- diversity: 1.2
----- Generating with seed: "
Ke

Edwardold
Owoon
Valne
narlan
Vly
Itezaz
Inlcal
Istarlas
Farrcho
Trroto
hevasdot
Beynoare
Bolly
Lansirt
HeFlLece
Lowoid
Kebman
Mimdie
Milbeis
Cllocka
Shial
Cllln
Clepkte
Zecl
Day
Incale
Diarea
Kiacllop
KClr
Epoch 91/100
6698/6698 [==============================] - 6s 837us/step - loss: 0.5301

----- diversity: 0.5
----- Generating with seed: "teban
Ethan
Ethelbert
Ethelred
Etienne
Euclid
Eugen
Eugene
Eustace
Ev
Evan
Evelyn
Everard
Everett
Ewan
Ewart
Ez
Ezechiel
Ezekiel
Ezra
Fabian
Fabio
Fairfax
Farley
Fazeel
Federico
Felice
Felicio
Felipe
"
teban
Ethan
Ethelbert
Ethelred
Etienne
Euclid
Eugen
Eugene
Eustace
Ev
Evan
Evelyn
Everard
Everett
Ewan
Ewart
Ez
Ezechiel
Ezekiel
Ezra
Fabian
Fabio
Fairfax
Farley
Fazeel
Federico
Felice
Felicio
Felipe
Fel
Weree
Trender
Gender
Geld
Gere
Gudree
Gie
GeldoGie
Ged
Redder
Ged
Redee
Rudder
Rudert
Reer
Rus
erd
Rere
Rod
Red
Rold
Rodai
Roddio
Robb
Rold
Rubeet
Rushey
Eug
Rednis
Arbey
Roes
Rdles
Reldonl
Role
R

----- diversity: 1.2
----- Generating with seed: "te

Odis
Todowoy
Eloo
Siedir
Sombio
Siskon
Scopsd
Sinpieph
Anis
Cynis
Conryin
Srinsy
Sispon
Siupin
Sigp
Iosi
Anofon
Doudont
Ruynin
gustongs
Sirven
Mil
Wartin
Garit
neray
Kessan
Hespan
Hedellmee

Halle
Saed
L
Epoch 97/100
6698/6698 [==============================] - 9s 1ms/step - loss: 0.4860

----- diversity: 0.5
----- Generating with seed: "k
Erin
Erl
Ernest
Ernesto
Ernie
Ernst
Erny
Errol
Ervin
Erwin
Esau
Esme
Esteban
Ethan
Ethelbert
Ethelred
Etienne
Euclid
Eugen
Eugene
Eustace
Ev
Evan
Evelyn
Everard
Everett
Ewan
Ewart
Ez
Ezechiel
Ezekie"
k
Erin
Erl
Ernest
Ernesto
Ernie
Ernst
Erny
Errol
Ervin
Erwin
Esau
Esme
Esteban
Ethan
Ethelbert
Ethelred
Etienne
Euclid
Eugen
Eugene
Eustace
Ev
Evan
Evelyn
Everard
Everett
Ewan
Ewart
Ez
Ezechiel
Ezekie
Feldy
Les
Lrcki
Herfald
Gedforndio
Grined
Rris
Mausied
Marmin
Marmo
MielomMphtum
Mognnice
Rud
Zener
chanee
Micke
MhicleMi
Weiker
Wilherm
WichiemWil
Ningel
Wiley
Willy
Wily
Chrlwis
Horsie
Herbert
Herr

----- diversity: 1.2
----- Generating with seed: "k
Erin

In [82]:
start_index = random.randint(0, len(text) - config_maxlen - 1)
start_index

13564

In [84]:
generated = ''
sentence = text[start_index: start_index + config_maxlen]
generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: "Pennie
Penny
Penrod
Pepe
Pepillo
Pepito
Perceval
Percival
Percy
Perry
Pete
Peter
Petey
Petr
Peyter
Peyton
Phil
Philbert
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter
"
Pennie
Penny
Penrod
Pepe
Pepillo
Pepito
Perceval
Percival
Percy
Perry
Pete
Peter
Petey
Petr
Peyter
Peyton
Phil
Philbert
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter


In [85]:
len(sentence)

200

In [86]:
sentence

'Pennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\n'

In [88]:
x_pred = np.zeros((1, config_maxlen, len(chars)))
for t, char in enumerate(sentence):
    x_pred[0, t, char_indices[char]] = 1.
x_pred

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]])

In [99]:
preds = model.predict(x_pred, verbose=0)[0]
preds

array([2.9561197e-04, 2.2416114e-05, 8.7929402e-06, 8.1856502e-03,
       4.0501584e-03, 1.3970521e-03, 9.1291520e-05, 5.3150332e-05,
       5.9026564e-03, 6.6148076e-04, 1.3645522e-04, 2.3192408e-02,
       1.1398835e-02, 2.8561231e-02, 1.5119918e-01, 8.3863111e-03,
       3.5496548e-04, 6.7535806e-01, 2.5483403e-05, 1.6705006e-04,
       2.4389757e-03, 5.8176345e-04, 2.4308389e-04, 5.3082354e-04,
       8.5133081e-03, 8.5963365e-03, 6.4706436e-04, 3.6507923e-02,
       2.4629784e-05, 3.5612477e-06, 2.0376236e-04, 1.1800623e-06,
       1.1756299e-06, 1.6263000e-05, 4.0886385e-05, 8.5403258e-03,
       8.1454091e-05, 2.3979681e-05, 9.7814796e-04, 9.9374158e-03,
       1.8650548e-05, 7.1890520e-07, 7.0984861e-08, 1.3957510e-03,
       1.1709863e-03, 2.4838917e-07, 2.5554332e-06, 2.6005939e-06,
       1.7735313e-07, 5.2413310e-07, 2.2801462e-06, 2.1710994e-05,
       3.5998878e-06, 1.9968456e-05], dtype=float32)

In [100]:
diversity = 0.5
next_index = sample(preds, diversity)
next_char = indices_char[next_index]
next_char

'P'

In [102]:
generated += next_char
generated

'Pennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\nP'

In [103]:
sentence

'Pennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\n'

In [104]:
sentence = sentence[1:] + next_char
sentence

'ennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\nP'